## Task: Question Answering for Game of Thrones

<img style="float: right;" src="https://upload.wikimedia.org/wikipedia/en/d/d8/Game_of_Thrones_title_card.jpg">

Question Answering can be used in a variety of use cases. A very common one:  Using it to navigate through complex knowledge bases or long documents ("search setting").

A "knowledge base" could for example be your website, an internal wiki or a collection of financial reports. 
In this tutorial we will work on a slightly different domain: "Game of Thrones". 

Let's see how we can use a bunch of wikipedia articles to answer a variety of questions about the 
marvellous seven kingdoms...  


In [ ]:
# Let's start by adjust the working directory so that it is the root of the repository
# This should be run just once.
import os
os.chdir('../')
print("Current working directory is {}".format(os.getcwd()))

In [ ]:
from haystack import Finder
from haystack.database.sql import SQLDocumentStore
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.io import write_documents_to_db, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.retriever.tfidf import TfidfRetriever
from haystack.utils import print_answers

## Indexing & cleaning documents

In [3]:
# Let's first get some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# The documents can be stored in different types of "DocumentStores".
# For dev we suggest a light-weight SQL DB
# For production we suggest elasticsearch
datastore = SQLDocumentStore(url="sqlite:///qa.db")

# Now, let's write the docs to our DB.
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
write_documents_to_db(datastore=datastore, document_dir=doc_dir, clean_func=clean_wiki_text, only_empty_db=True)

11/28/2019 12:02:51 - INFO - haystack.indexing.io -   Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip to `data/article_txt_got`
100%|██████████| 1167348/1167348 [00:00<00:00, 8157729.91B/s]
11/28/2019 12:02:52 - INFO - haystack.indexing.io -   Wrote 517 docs to DB


## Initalize Reader, Retriever & Finder

In [4]:
# A retriever identifies the k most promising chunks of text that might contain the answer for our question
# Retrievers use some simple but fast algorithm, here: TF-IDF
retriever = TfidfRetriever(datastore=datastore)

11/28/2019 12:02:56 - INFO - haystack.retriever.tfidf -   Found 2813 candidate paragraphs from 519 docs in DB


In [5]:
# A reader scans the text chunks in detail and extracts the k best answers
# Reader use more powerful but slower deep learning models
# You can select a local model or  any of the QA models published on huggingface's model hub (https://huggingface.co/models)
# here: a medium sized BERT QA model trained via FARM on Squad 2.0
reader = FARMReader(model_name_or_path="deepset/bert-base-cased-squad2", use_gpu=False)

# OR: use alternatively a reader from huggingface's transformers package (https://github.com/huggingface/transformers)
# reader = TransformersReader(model="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)

11/28/2019 12:02:57 - INFO - haystack.indexing.io -   Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-models/0.3.0/bert-english-qa-large.tar.gz to `model`
100%|██████████| 1245547135/1245547135 [00:53<00:00, 23191957.12B/s]
11/28/2019 12:04:05 - WARNING - farm.data_handler.processor -   Loading tokenizer from deprecated FARM config. If you used `custom_vocab` or `never_split_chars`, this won't work anymore.


In [6]:
# The Finder sticks together retriever and retriever in a pipeline to answer our actual questions
finder = Finder(reader, retriever)

## Voilá! Ask a question!

In [7]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
prediction = finder.get_answers(question="Who is the father of Arya Stark?", top_k_retriever=10, top_k_reader=5)

11/28/2019 12:04:26 - INFO - haystack -   Identified 10 candidates via retriever:
  paragraph_id  document_id                                                                             text
         2723          506  \n===Arya Stark===\n'''Arya Stark''' portrayed by Maisie Williams. Arya Star...
         2212          407  \n====Season 8====\nArya reunites with Jon, Gendry, and the Hound, who have ...
         2205          407  \n====Season 1====\nArya accompanies her father Ned and her sister Sansa to ...
          548          105  \n===''A Game of Thrones''===\nSansa Stark begins the novel by being betroth...
         1437          258  \n===In Braavos===\nLady Crane returns to her chambers to find a wounded Ary...
          462           92  \n== Characters ==\nThe tale is told through the eyes of 9 recurring POV cha...
         2198          407  \n==== ''A Game of Thrones'' ====\nArya adopts a direwolf cub, which she nam...
         2211          407  \n====Season 7====\nTakin

In [8]:
#prediction = finder.get_answers(question="Who created the Dothraki vocabulary?", top_k_reader=5)
#prediction = finder.get_answers(question="Who is the sister of Sansa?", top_k_reader=5)

In [8]:
print_answers(prediction, details="minimal")

[   {   'answer': 'Eddard',
        'context': 'ry warrior queen. She travels with her father, Eddard, to '
                   "King's Landing when he is made Hand of the"},
    {   'answer': 'Ned',
        'context': '\n'
                   '====Season 1====\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King's Landing. Before the"},
    {   'answer': 'Lord Eddard',
        'context': ' is the younger daughter and third child of Lord Eddard '
                   'and Catelyn Stark of Winterfell. Ever the to'},
    {   'answer': 'Lord Eddard Stark',
        'context': ' Tourney of the Hand to honour her father Lord Eddard '
                   'Stark, Sansa Stark is enchanted by the knights'},
    {   'answer': 'Eddard and Catelyn Stark',
        'context': 'e third child and younger daughter of Eddard and Catelyn '
                   'Stark. She serves as a POV character for 33'}]
